# Conformance Checking (DECLARE Formulas) in Declare4Py

This tutorial explains how to perform the checking of a DECLARE model and how to browse the results. We start by importing the classes for managing with DECLARE models: `DeclareModel` and `DeclareModelTemplate`:

In [1]:
import sys
import os
import pathlib

SCRIPT_DIR = pathlib.Path("..", "src").resolve()
sys.path.append(os.path.dirname(SCRIPT_DIR))

from src.Declare4Py.ProcessModels.DeclareModel import DeclareModel
from src.Declare4Py.ProcessModels.DeclareModel import DeclareModelTemplate

The DECLARE constraints supported by Declare4Py can be retrieved with the `get_unary_templates()` and `get_binary_templates` functions of the `DeclareModelTemplate` class:

In [2]:
unary_templates = DeclareModelTemplate.get_unary_templates()
binary_templates = DeclareModelTemplate.get_binary_templates()

print("Unary templates:")
print("-----------------")
for template in unary_templates:
    print(template.templ_str)
print("\n")

print("Binary templates:")
print("-----------------")
for template in binary_templates:
    print(template.templ_str)

Unary templates:
-----------------
Existence
Absence
Exactly
Init


Binary templates:
-----------------
Choice
Exclusive Choice
Responded Existence
Response
Alternate Response
Chain Response
Precedence
Alternate Precedence
Chain Precedence
Succession
Alternate Succession
Co-Existence
Chain Succession
Not Chain Succession
Not Co-Existence
Not Succession
Not Responded Existence
Not Response
Not Precedence
Not Chain Response
Not Chain Precedence


Notice that for the templates `Existence`, `Absence` and `Exactly` an additional parameter is necessary for the cardinality. This has to be encoded in the DECLARE `.decl` model with a numeric suffix, for example `Exactly2` or `Existence23`.

The next step is the parsing of the log and of the DECLARE model.

In [3]:
from src.Declare4Py.D4PyEventLog import D4PyEventLog

log_path = os.path.join("..", "tests", "Sepsis Cases.xes.gz")
event_log = D4PyEventLog(case_name="case:concept:name")
event_log.parse_xes_log(log_path)

model_path = os.path.join("..", "tests", "resource", "files", "declare", "data_model.decl")
declare_model = DeclareModel().parse_from_file(model_path)

parsing log, completed traces ::   0%|          | 0/1050 [00:00<?, ?it/s]

The DECLARE model can be inspected by getting all the activity names or the constraints. This information is returned as a list of strings.

In [4]:
model_activities = declare_model.get_model_activities()
model_constraints = declare_model.get_decl_model_constraints()

print("Model activities:")
print("-----------------")
for idx, act in enumerate(model_activities):
    print(idx, act)
print("\n")

print("Model constraints:")
print("-----------------")
for idx, constr in enumerate(model_constraints):
    print(idx, constr)

Model activities:
-----------------
0 ER Triage
1 ER Registration
2 ER Sepsis Triage
3 Leucocytes
4 CRP
5 LacticAcid
6 IV Antibiotics
7 Admission NC
8 IV Liquid
9 Release A
10 Return ER
11 Admission IC


Model constraints:
-----------------
0 Response[ER Registration, Leucocytes] | | |
1 Chain Response[ER Registration, ER Triage] |(A.DiagnosticArtAstrup is false) AND (A.SIRSCritHeartRate is true) AND (A.org:group is A) AND (A.DiagnosticBlood is true) AND (A.DisfuncOrg is false) AND (A.DiagnosticECG is true) AND (A.Age >= 45) AND (A.InfectionSuspected is true) AND (A.DiagnosticLacticAcid is true) AND (A.DiagnosticSputum is true) AND (A.Hypoxie is false) AND (A.DiagnosticUrinaryCulture is true) AND (A.DiagnosticLiquor is false) AND (A.SIRSCritTemperature is true) AND (A.Infusion is true) AND (A.Hypotensie is false) AND (A.DiagnosticUrinarySediment is true) AND (A.Oligurie is false) AND (A.Age <= 80) AND (A.SIRSCritTachypnea is true) AND (A.DiagnosticOther is false) AND (A.SIRSCritLeucos 

We import and instantiate the class `MPDeclareAnalyzer` to set the DECLARE conformance checking (Declare4Py implements the MP DECLARE analyzer algorithm). The `MPDeclareAnalyzer` constructor takes as an input the boolean parameter `consider_vacuity=true` that means that vacuously satisfied traces are considered as satisfied, violated otherwise. The constructor also needs the `event_log` and the `declare_model` objects. After this setting, we execute the method `run` of the `MPDeclareAnalyzer` class.

In [5]:
from src.Declare4Py.ProcessMiningTasks.ConformanceChecking.MPDeclareAnalyzer import MPDeclareAnalyzer
from src.Declare4Py.ProcessMiningTasks.ConformanceChecking.MPDeclareResultsBrowser import MPDeclareResultsBrowser


basic_checker = MPDeclareAnalyzer(log=event_log, declare_model=declare_model, consider_vacuity=False)
conf_check_res: MPDeclareResultsBrowser = basic_checker.run()

The result of `basic_checker.run()` is a `ResultsBrowser` object that allows for the retrieval of the conformance checking results with the `get_metric()` method. This takes as input the `metric` parameter with values in `num_pendings`, `num_activations`, `num_fulfillments`, `num_violations` and `state`. This return a table whose rows are the results of each trace according to the DECLARE constraints in the model (as columns). For example, this execution

In [6]:
conf_check_res.get_metric(metric="num_activations")

,"Response[ER Registration, Leucocytes] | | |","Chain Response[ER Registration, ER Triage] |(A.DiagnosticArtAstrup is false) AND (A.SIRSCritHeartRate is true) AND (A.org:group is A) AND (A.DiagnosticBlood is true) AND (A.DisfuncOrg is false) AND (A.DiagnosticECG is true) AND (A.Age >= 45) AND (A.InfectionSuspected is true) AND (A.DiagnosticLacticAcid is true) AND (A.DiagnosticSputum is true) AND (A.Hypoxie is false) AND (A.DiagnosticUrinaryCulture is true) AND (A.DiagnosticLiquor is false) AND (A.SIRSCritTemperature is true) AND (A.Infusion is true) AND (A.Hypotensie is false) AND (A.DiagnosticUrinarySediment is true) AND (A.Oligurie is false) AND (A.Age <= 80) AND (A.SIRSCritTachypnea is true) AND (A.DiagnosticOther is false) AND (A.SIRSCritLeucos is false) AND (A.DiagnosticIC is true) AND (A.SIRSCriteria2OrMore is true) AND (A.DiagnosticXthorax is true) |T.org:group is C |52,2154,s",Existence2[Admission NC] | |,"Chain Response[Admission NC, Release B] |A.org:group is K |T.org:group is E |","Chain Response[Admission NC, Release A] |A.org:group is I |T.org:group is E |133020,957701,s","Chain Precedence[IV Liquid, Admission NC] |A.org:group is I |T.org:group is A |92,14473,s","Chain Precedence[Release A, Return ER] |A.org:group is ? |T.org:group is E |1121801,1121801,s","Chain Precedence[ER Sepsis Triage, IV Antibiotics] |A.org:group is L |T.org:group is L |15,11000,s","Chain Response[ER Sepsis Triage, IV Antibiotics] |A.org:group is L |T.org:group is L |15,11000,s","Chain Precedence[Admission IC, Admission NC] |A.org:group is J |T.org:group is J |","Chain Precedence[IV Antibiotics, Admission NC] |A.org:group is F |T.org:group is A |92,14459,s","Chain Precedence[Admission NC, Release B] |A.org:group is E |T.org:group is K |48225,48225,s","Chain Response[Admission IC, Admission NC] |A.org:group is J |T.org:group is J |61534,61534,s","Chain Response[LacticAcid, Leucocytes] |A.LacticAcid <= 0.8 |T.Leucocytes >= 13.8 |0,2778,m","Chain Precedence[ER Registration, ER Triage] |A.org:group is C |(T.InfectionSuspected is true) AND (T.SIRSCritTemperature is true) AND (T.DiagnosticLacticAcid is true) AND (T.DiagnosticBlood is true) AND (T.DiagnosticIC is true) AND (T.SIRSCriteria2OrMore is true) AND (T.DiagnosticECG is true) |52,2154,s"
0,1,0,None,0,0,0,0,0,0,0,0,0,0,0,1
1,1,0,None,0,0,0,0,0,0,0,1,0,0,1,1
2,1,0,None,0,0,0,0,0,0,0,0,0,0,0,1
3,1,0,None,0,0,0,1,0,0,0,1,0,0,0,1
4,1,0,None,0,0,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1045,1,0,None,0,0,0,0,0,0,0,1,0,0,0,1
1046,1,0,None,0,0,0,0,0,0,0,0,0,0,0,1
1047,1,0,None,0,0,0,0,0,0,0,0,0,0,0,1
1048,1,0,None,0,0,0,0,0,0,0,0,0,0,1,1


returns the number of activations of each trace (rows) for each column. We can also specify a `trace_id`, a `constraint_id` or both (we see an example with the constrain `Chain Response[LacticAcid, Leucocytes] |A.LacticAcid <= 0.8 |T.Leucocytes >= 13.8 |0,2778,m` at index 13):

In [7]:
# Number of activations for the second trace
print(conf_check_res.get_metric(trace_id=1, metric="num_activations"))
print("-------------------------------------------")

# Number of activations for the 13th constraint
print(conf_check_res.get_metric(constr_id=13, metric="num_activations"))
print("-------------------------------------------")

# Number of activations for the second trace and the 13th constraint
conf_check_res.get_metric(trace_id=1, constr_id=13, metric="num_activations")

[1, 0, None, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1]
-------------------------------------------
[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 12, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

1

Similar retrievals can be done for the other metrics. For example, the `state` metric return 1 if the constraint is satisfied in the trace and 0 otherwise:

In [8]:
# Truth values for the second trace
print(conf_check_res.get_metric(trace_id=1, metric="state"))
print("-------------------------------------------")

# Truth values for the 13th constraint
print(conf_check_res.get_metric(constr_id=13, metric="state"))
print("-------------------------------------------")

# Truth value for the second trace and the 13th constraint
print(conf_check_res.get_metric(trace_id=1, constr_id=13, metric="state"))

[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0]
-------------------------------------------
[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 